In [5]:
import pandas as pd
import numpy as np

In [32]:
def get_var(file, trait, e):
    with open(file) as f:
        current_e_flag=False
        model_PRSxE=np.nan
        model_PRS=np.nan
        model_E=np.nan
        model_PRSxE_pval = np.nan
        all_VE = np.nan
        for line in f:
            l = line.rstrip().split()
            if trait in line and e in line:
                current_e_flag=True
                all_VE = 0
                continue
            if current_e_flag:
                try:
                    all_VE += float(l[-1])
                except (IndexError, ValueError):
                    continue
                if l[0] == 'SCORE1_AVG':
                    model_PRS = float(l[-1])
                if l[0] == 'VAR':
                    model_E = float(l[-1])
                if l[0] == 'SCORE1_AVG:VAR':
                    model_PRSxE_beta = float(l[-4])
                    model_PRSxE_pval = float(l[-2])
                if l[0] == 'PC10_49K':
                    current_e_flag=False
        return(model_PRSxE_pval, model_PRSxE_beta)

In [33]:
def compute_PRSxE_VE(trait, e):
    file='/Users/arun/Documents/pricelab/ps_gxe/2023_05_10_tables/data/regression_update/'+trait+'.txt'
    interaction = get_var(file, trait, e)
    file='/Users/arun/Documents/pricelab/ps_gxe/2023_05_10_tables/data/regression_update/'+trait+'_noE2.txt'
    no_E2 = get_var(file, trait, e)
    return([*interaction, *no_E2])

In [34]:
traits = list(pd.read_csv("/Users/arun/Documents/pricelab/ps_gxe/2023_05_10_tables/data/data_update/traits.txt", header=None)[0])
E_vars = list(pd.read_csv("/Users/arun/Documents/pricelab/ps_gxe/2023_05_10_tables/data/data_update/E.txt", header=None)[0])

In [37]:
with open("/Users/arun/Documents/pricelab/ps_gxe/2023_05_10_tables/parsed/PRS_noE2_E2.csv", 'w') as f:
    for t in traits:
        for e in E_vars:
            print(t, e, *compute_PRSxE_VE(t, e), sep=',', file=f)